# HIDDEN MARKOV NEURAL NETWORK: MNIST example

## Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn import preprocessing


import pickle

import gzip

import numpy as np
import random

import BayesianNetwork

## Data preprocessing

In [2]:
# Set the seed to guarantee reproducibility
seed_number = 123

random.seed(seed_number)
torch.manual_seed(seed_number)
np.random.seed(seed_number)

The files "train-labels-idx1-ubyte.gz" and "train-images-idx3-ubyte.gz" are downloaded from:

- http://yann.lecun.com/exdb/mnist/

In [3]:
def load_mnist():
    """Load MNIST data from `path`"""
    labels_path = 'train-labels-idx1-ubyte.gz'
    images_path = 'train-images-idx3-ubyte.gz'
        
    with gzip.open(labels_path, 'rb') as lbpath:
        lbpath.read(8)
        buffer = lbpath.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        imgpath.read(16)
        buffer = imgpath.read()
        images = np.frombuffer(buffer, 
                               dtype=np.uint8).reshape(
            len(labels), 784).astype(np.float64)
 
    return images, labels

def mnist_preprocessing(x, y, sample_N = 600000, test_ratio = 0.25):

    x = np.float32(x) / 126.
    np.save("mnist_preprocessed_data", x)
    y = np.int32(y)
    np.save("mnist_preprocessed_target", y)
    idx = np.random.choice(x.shape[0], sample_N)
    x = x[idx]
    y = y[idx]

    tr_idx, te_idx = train_test_split(np.arange(sample_N), test_size = test_ratio)
    tr_x, te_x = x[tr_idx], x[te_idx]
    tr_y, te_y = y[tr_idx], y[te_idx]

    return tr_x,te_x,tr_y,te_y

In [4]:
x, y = load_mnist()
tr_x, va_x, tr_y, va_y = mnist_preprocessing(x, y)

# Split in training and validation: 
x_tr  = tr_x[0:50000]
y_tr  = tr_y[0:50000]

x_val = va_x[50000:60000]
y_val = va_y[50000:60000]

## Set the hyper parameters for the training

In [5]:
# The sample size
sample_size    = 10000
# The minibatch size
minibatch_size = 128
# The number of epochs
epocs          = 600
# This parameter if use to retrain on part of the previous data. 
# If sliding = sample_size then we move to a new set of data.
sliding = 10000
# Number of Sequential training we want to do
T = 5

###########################################################
# Set the network structure
# Depth
L = 4
# Structure of the hidden units
architecture = np.array([784, 400, 400, 10])

# Kernel parameter of the HMNN
alpha_k = 0.25
sigma_k = np.exp(2)
c       = np.exp(10)
pi      = 0.5

# Mixture weight for the variational dropconnect
p       = 0.3

# Learning rate
lr_c = 1e-3

# Size of the Monte Carlo sample
mc_c = 1

## Training procedure

In [ ]:
# Loss function
loss_function = torch.nn.CrossEntropyLoss(reduction = 'sum')


HMMNET = BayesianNetwork.torchHHMnet(architecture, alpha_k, sigma_k, c, pi, p, loss_function, sample_size, minibatch_size, epocs, T, sliding, workers = 4)

HMMNET.forward_pass(x_tr, y_tr, x_val, y_val, lr_c, mc_c)

Time  1
Epoch  1
Prior score  19707.05003712239  and Data score  36.713829778182976
Performance on the validation set  0.1329
Epoch  2
Prior score  19698.501097626573  and Data score  36.18250173923501
Performance on the validation set  0.1024
Epoch  3
Prior score  19703.18217511245  and Data score  34.659096004751966
Performance on the validation set  0.1041
Epoch  4
Prior score  19637.42988134435  and Data score  35.19201698128595
Performance on the validation set  0.1507
Epoch  5
Prior score  19622.080439713845  and Data score  35.067765606971236
Performance on the validation set  0.4107
Epoch  6
Prior score  19643.210363773826  and Data score  25.84581446183535
Performance on the validation set  0.5509
Epoch  7
Prior score  19633.343093588814  and Data score  19.84940612129417
Performance on the validation set  0.6961
Epoch  8
